<a href="https://colab.research.google.com/github/s-c-soma/deep_context_graph/blob/main/code/TrendDetection_neurips_conf_extract_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scrape Neurips Data

## Libraries

In [1]:
!pip install dateparser

     |████████████████████████████████| 286kB 6.7MB/s 


In [2]:
#Import the dependencies
from bs4 import BeautifulSoup
import pandas as pd
import requests
import urllib.request
import time
from tqdm import tqdm
from dateparser.search import search_dates
import re
from abc import ABC, abstractmethod
from typing import List

## Init

In [35]:
#Create lists to store the scraped data
#type	title	authors	abstract	category	keywords	url

sources = []
urls = []
titles = []
summaries = []
dates = []
ratings = []
bodies = []
authors= []
publishdate = []
relatedlinks = []
claims = []
abstracts = []
type_ = []
MATCH_ALL = r'.*'

## Methods

In [36]:
def like(string):
    """
    Return a compiled regular expression that matches the given
    string with any prefix and postfix, e.g. if string = "hello",
    the returned regex matches r".*hello.*"
    """
    string_ = string
    if not isinstance(string_, str):
        string_ = str(string_)
    regex = MATCH_ALL + re.escape(string_) + MATCH_ALL
    return re.compile(regex, flags=re.DOTALL)

In [37]:
def find_by_text(soup, text, tag, **kwargs):
    """
    Find the tag in soup that matches all provided kwargs, and contains the
    text.

    If no match is found, return None.
    If more than one match is found, raise ValueError.
    """
    elements = soup.find_all(tag, **kwargs)
    matches = []
    for element in elements:
        if element.find(text=like(text)):
            matches.append(element)
    if len(matches) == 0:
        return None

    return matches

In [38]:
def extract_claim_and_review(parsed_claim_review_page, url):
        urls.append(url)
        sources.append("washingtonpost")

        
        # title
        title = parsed_claim_review_page.find("h1", {"class": " font--headline gray-darkest pb-sm null "})
        #print("title", title.text)
        #soup = BeautifulSoup(title.text)
        #print("title", soup.text)
        titles.append(title.text)
        

        # date
        date_ = parsed_claim_review_page.find('div', {"class": "display-date "})#.find("p")
        #print("date_", date_)
        if date_:
            date_str = search_dates(date_.text)[0][1].strftime("%Y-%m-%d")
            #print("url_date", url_date)
            dates.append(date_str)

        # body
        body = parsed_claim_review_page.find("div", {"class": "article-body"})
        #print("body=", body.get_text())
        #claim.set_body(body.get_text())
        bodies.append(body.get_text())
        

        # related links
        divTag = parsed_claim_review_page.find("div", {"class": "article-body"})
        related_links = []
        for link in divTag.findAll('a', href=True):
            related_links.append(link['href'])
        relatedlinks.append(related_links)

     
        #claims
        tags = []
        for tag in parsed_claim_review_page.findAll('meta', {"property": "article:tag"}):
            tags.append(tag["content"])
        claims.append(", ".join(tags))
       
        return None

In [39]:
 def extract_urls(parsed_listing_page):
        urls = list()
        titles = list()
        authors = list()
        # links = parsed_listing_page.find('ui').findAll('a', href=True)
        # finding all li tags in ul and printing the text within it
        body = soup.find("div", {"class": "col"})
        data1 = body.find('ul')
        #print(data1)
        for li in data1.findAll('a', href=True): 
            url = "https://papers.nips.cc" + str(li['href'])
            max_claims = 0
            if 0 < max_claims <= len(urls):
                break
            #if url not in self.configuration.avoid_urls:
            urls.append(url)
            titles.append(li.text)
        for li in data1.findAll('i'): 
            #print(li)
            authors.append(li)
            #break
        return urls,titles,authors  

In [40]:
def extract_abstract(parsed_listing_page):
    sources.append("neurips")
    type_.append("conference")

    abstracts = list()
    body = parsed_listing_page.find("div", {"class": "col"})
    p_tags = body.find_all(["p"])
    
    if len(p_tags)>3:
      #print(p_tags[3].text)
      abstracts.append(p_tags[3].text)
    else: 
      #print(p_tags)
      abstracts.append(p_tags.text)

    

In [41]:
abstracts = []

In [42]:
def extract_abstract2(soup):
    sources.append("neurips")
    type_.append("conference")

    #abstracts = list()
    body = soup.find("div", {"class": "col"})
    p_tags = body.find_all(["p"])
    
    body = soup.find("div", {"class": "col"})
    #body
    p_tags = body.find_all(["p"])
    if len(p_tags)>3:
      #print(p_tags[3].text)
      abstracts.append(p_tags[3])
    else: 
      abstracts.append(p_tags)
      #print(p_tags)

## Level 1: Scrape Urls

In [43]:
page_number = 2
URL = 'https://papers.nips.cc/paper/2020'
webpage = requests.get(URL)  #Make a request to the website
soup = BeautifulSoup(webpage.text, "html.parser")
#print(soup.prettify())

extract_url,titles, authors = extract_urls(soup)
extract_url

['https://papers.nips.cc/paper/2020/hash/0004d0b59e19461ff126e3a08a814c33-Abstract.html',
 'https://papers.nips.cc/paper/2020/hash/00482b9bed15a272730fcb590ffebddd-Abstract.html',
 'https://papers.nips.cc/paper/2020/hash/0060ef47b12160b9198302ebdb144dcf-Abstract.html',
 'https://papers.nips.cc/paper/2020/hash/007ff380ee5ac49ffc34442f5c2a2b86-Abstract.html',
 'https://papers.nips.cc/paper/2020/hash/0084ae4bc24c0795d1e6a4f58444d39b-Abstract.html',
 'https://papers.nips.cc/paper/2020/hash/00a03ec6533ca7f5c644d198d815329c-Abstract.html',
 'https://papers.nips.cc/paper/2020/hash/00ac8ed3b4327bdd4ebbebcb2ba10a00-Abstract.html',
 'https://papers.nips.cc/paper/2020/hash/00e26af6ac3b1c1c49d7c3d79c60d000-Abstract.html',
 'https://papers.nips.cc/paper/2020/hash/012a91467f210472fab4e11359bbfef6-Abstract.html',
 'https://papers.nips.cc/paper/2020/hash/012d9fe15b2493f21902cd55603382ec-Abstract.html',
 'https://papers.nips.cc/paper/2020/hash/0163cceb20f5ca7b313419c068abd9dc-Abstract.html',
 'https://

## Level 2: Scrape Abstracts from 1st Level Urls

In [30]:
print(len(extract_url))

1898


In [44]:

#extract_url = ['https://papers.nips.cc/paper/2020/hash/012d9fe15b2493f21902cd55603382ec-Abstract.html']
for URL in extract_url:
  #print(URL)
  webpage = requests.get(URL)
  soup = BeautifulSoup(webpage.text, "html.parser") #Parse the text from the website
  #print(soup.prettify())
  extract_abstract2(soup)
  #print("****************************")

## Dataframe

In [45]:
abstracts

[<p>Graph neural networks (GNNs) have been successful in learning representations from graphs. Many popular GNNs follow the pattern of <em>aggregate-transform</em>: they aggregate the neighbors' attributes and then transform the results of aggregation with a learnable function. Analyses of these GNNs explain which pairs of non-identical graphs have different representations. However, we still lack an understanding of how similar these representations will be. We adopt kernel distance and propose <em>transform-sum-cat</em> as an alternative to aggregate-transform to reflect the continuous similarity between the node neighborhoods in the neighborhood aggregation. The idea leads to a simple and efficient graph similarity, which we name Weisfeiler-Leman similarity (WLS). In contrast to existing graph kernels, WLS is easy to implement with common deep learning frameworks. In graph classification experiments, transform-sum-cat significantly outperforms other neighborhood aggregation methods 

In [46]:
#Create a new dataFrame 
data = pd.DataFrame(columns = ['type', 'sources', 'titles', 'authors', 'abstract', 'urls']) 
data['type'] = type_
data['sources'] = sources
data['titles'] = titles
data['authors'] = authors
data['abstract'] = abstracts
data['urls'] = extract_url

#Show the data set
data

,type,sources,titles,authors,abstract,urls
0,conference,neurips,A graph similarity for deep learning,[Seongmin Ok],[Graph neural networks (GNNs) have been succes...,https://papers.nips.cc/paper/2020/hash/0004d0b...
1,conference,neurips,An Unsupervised Information-Theoretic Perceptu...,"[Sangnie Bhardwaj, Ian Fischer, Johannes Ballé...",[Tractable models of human perception have pro...,https://papers.nips.cc/paper/2020/hash/00482b9...
2,conference,neurips,Self-Supervised MultiModal Versatile Networks,"[Jean-Baptiste Alayrac, Adria Recasens, Rosali...",[Videos are a rich source of multi-modal super...,https://papers.nips.cc/paper/2020/hash/0060ef4...
3,conference,neurips,"Benchmarking Deep Inverse Models over time, an...","[Simiao Ren, Willie Padilla, Jordan Malof]",[We consider the task of solving generic inver...,https://papers.nips.cc/paper/2020/hash/007ff38...
4,conference,neurips,Off-Policy Evaluation and Learning for Externa...,"[Masatoshi Uehara, Masahiro Kato, Shota Yasui]",[We consider the evaluation and training of a ...,https://papers.nips.cc/paper/2020/hash/0084ae4...
...,...,...,...,...,...,...
1893,conference,neurips,Distributed Distillation for On-Device Learning,"[Ilai Bistritz, Ariana Mann, Nicholas Bambos]",[On-device learning promises collaborative tra...,https://papers.nips.cc/paper/2020/hash/fef6f97...
1894,conference,neurips,COOT: Cooperative Hierarchical Transformer for...,"[Simon Ging, Mohammadreza Zolfaghari, Hamed Pi...",[Many real-world video-text tasks involve diff...,https://papers.nips.cc/paper/2020/hash/ff0abbc...
1895,conference,neurips,Passport-aware Normalization for Deep Model Pr...,"[Jie Zhang, Dongdong Chen, Jing Liao, Weiming ...",[Despite tremendous success in many applicatio...,https://papers.nips.cc/paper/2020/hash/ff1418e...
1896,conference,neurips,Sampling-Decomposable Generative Adversarial R...,"[Binbin Jin, Defu Lian, Zheng Liu, Qi Liu, Jia...",[Recommendation techniques are important appro...,https://papers.nips.cc/paper/2020/hash/ff42b03...


In [47]:
import os
# Mount google drive
from google.colab import drive
drive.mount('/content/drive',force_remount=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [48]:
file_path = '/content/drive/Shared drives/1DeepContextGraph/1DeepContextGraph/code/data/'

In [49]:
data.to_csv(file_path+'neurips_data_2020.csv', index=False)

# Testing

In [ ]:
url = ['https://papers.nips.cc/paper/2020/hash/012d9fe15b2493f21902cd55603382ec-Abstract.html']
webpage = requests.get(URL)
soup = BeautifulSoup(webpage.text, "html.parser") #Parse the text from the website
#print(soup.prettify())

extract_abstract2(soup)


body = soup.find("div", {"class": "col"})
body
p_tags = body.find_all(["p"])
if len(p_tags)>3:
  print(p_tags[3].text)
else: 
  print(p_tags)

#print(p_tags)
#print(p_tags[3].text)
